In [15]:
import numpy as np

import astropy.units as u
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
from astropy.table import Table, Column
from astropy.io import ascii
from astropy.time import Time
from astropy.coordinates import SkyCoord, Distance

In [2]:
table = Gaia.load_table('gaiaedr3.gaia_source')

Retrieving table 'gaiaedr3.gaia_source'


In [3]:
SPEC_CAT = ascii.read("../reference_data/color_table.txt").to_pandas()

def gaia_sptype(color, cat=SPEC_CAT, color_name='G-Rp'):
    diff = cat[color_name] - color
    absdiff = abs(diff)
    sptype = cat.loc[absdiff.idxmin()]['SpT'].replace('V', '')
    return sptype

In [4]:
for col in table.columns:
    print(col.name)

solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pm
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
nu_eff_used_in_astrometry
pseudocolour
pseudocolour_error
ra_pseudocolour_corr
dec_pseudocolour_corr
parallax_pseudocolour_corr
pmra_pseudocolour_corr
pmdec_pseudocolour_corr
astrometric_matched_transits
visibility_periods_used
astrometric_sigma5d_max
matched_transits
new_matched_transits
matched_transits_removed
ipd_gof_harmonic_amplitude
ipd_gof_harmonic_phase
ipd_frac_multi_peak
ipd_frac_odd_win
ruwe
scan_direction_strength_k1
scan_di

In [5]:
def gaia2mmt(low_mag, high_mag, every=10, outfile=None):
    """
    Grab stars from Gaia EDR3 in the range between low_mag and high_mag. The 'every' parameter can be used to 
    limit the size of the sample, e.g. to select every 10th star. This uses the random_index in the Gaia table to
    assure sampling is representative. For MMT catalogs, about 15000-20000 entries is about the limit of what the
    scope gui can comfortably handle. The output table is configured to output an MMT-compatible catalog directly. 
    If outfile is not None, the catalog will be written to outfile. 
    """

    job = Gaia.launch_job_async(
        "select all designation,ref_epoch,ra,dec,pmra,pmdec,phot_g_mean_mag as gmag,g_rp,random_index "
        f"from gaiaedr3.gaia_source where (phot_g_mean_mag > {low_mag} AND phot_g_mean_mag < {high_mag} AND dec > -40 AND "
        f"pmra IS NOT NULL AND MOD(random_index, {every}) = 0) order by gmag"
    )
    r = job.get_results()
    r['sptype'] = [gaia_sptype(g_rp) for g_rp in r['g_rp']]
    ids = [f"GEDR3_{s.split()[-1]}" for s in r['designation']]
    t_obs = Time(r['ref_epoch'].data, format='decimalyear')
    t_ref = Time('J2000')
    c = SkyCoord(
        ra=r['ra'],
        dec=r['dec'],
        pm_ra_cosdec=r['pmra'],
        pm_dec=r['pmdec'],
        obstime=t_obs,
        frame='icrs'
    )
    c_j2000 = c.apply_space_motion(t_ref)
    ra_2000 = c_j2000.ra.to_string(unit=u.hr, sep=":", precision=4, pad=True)
    dec_2000 = c_j2000.dec.to_string(unit=u.deg, sep=":", alwayssign=True, precision=3, pad=True)
    pmra_mmt = r['pmra'].data / 15. / 10. / np.cos(c_j2000.dec)  # MMT uses seconds of time per century
    pmdec_mmt = r['pmdec'].data / 10.  # MMT uses seconds of arc per century

    out_tab = Table()
    out_tab['ID'] = Column(ids, format='<25')
    out_tab['RA'] = Column(ra_2000)
    out_tab['Dec'] = Column(dec_2000)
    out_tab['PM_RA'] = Column(pmra_mmt.data, format=".4f")
    out_tab['PM_Dec'] = Column(pmdec_mmt.data, format=".4f")
    out_tab['Mag'] = Column(r['gmag'].data, format=".2f")
    out_tab['Sp_Type'] = Column(r['sptype'])
    out_tab.add_column('J2000.0', name='Epoch')
    if outfile is not None:
        out_tab.write(outfile, format='ascii.fixed_width_no_header', delimiter=None, overwrite=True, bookend=False)
    return out_tab

In [10]:
t = gaia2mmt(low_mag=7, high_mag=8, every=1, outfile="../catalogs/gaia_7_8.cat")

INFO: Query finished. [astroquery.utils.tap.core]


/home/tim/conda/envs/py39/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 32054 of "distance overridden (Note 6)", 2 of "binary logical OR of the above warnings"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [12]:
bright = gaia2mmt(low_mag=-2, high_mag=7, every=1)

INFO: Query finished. [astroquery.utils.tap.core]


/home/tim/conda/envs/py39/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 16360 of "distance overridden (Note 6)", 3 of "binary logical OR of the above warnings"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [13]:
bright

ID,RA,Dec,PM_RA,PM_Dec,Mag,Sp_Type,Epoch
str25,str13,str13,float64,float64,float32,str4,str7
GEDR3_418551920284673408,00:40:30.4431,+56:32:14.385,0.5940,-3.1595,1.94,A3,J2000.0
GEDR3_4357027756659697664,16:14:20.7390,-03:41:39.573,-0.3029,-14.4433,2.02,A3,J2000.0
GEDR3_5589311357728452608,07:17:08.5581,-37:05:50.904,-0.0966,0.6520,2.08,A7,J2000.0
GEDR3_4038055447778237312,18:17:37.6372,-36:45:42.085,-1.0968,-16.6309,2.12,A8,J2000.0
GEDR3_1279798794197267072,14:44:59.2176,+27:04:27.200,-0.3805,2.1024,2.18,A3,J2000.0
GEDR3_160886283751041408,04:56:59.6200,+33:09:57.938,0.0304,-1.7786,2.20,A6,J2000.0
GEDR3_4302054339959905920,19:46:15.5790,+10:36:47.731,0.1101,-0.1354,2.22,F4,J2000.0
GEDR3_1222646935698492160,15:34:41.2685,+26:42:52.866,0.8876,-8.7711,2.27,B9,J2000.0
GEDR3_5111187420714898304,03:58:01.7658,-13:30:30.653,0.4266,-11.2006,2.27,F5,J2000.0


In [18]:
Simbad.TIMEOUT = 240
ids = [s.replace('GEDR3_', "Gaia EDR3 ") for s in bright['ID'].data]
sim_table = Simbad.query_objects_async(ids)
sim_table

<Response [200]>

In [21]:
ids

['Gaia EDR3 418551920284673408',
 'Gaia EDR3 4357027756659697664',
 'Gaia EDR3 5589311357728452608',
 'Gaia EDR3 4038055447778237312',
 'Gaia EDR3 1279798794197267072',
 'Gaia EDR3 160886283751041408',
 'Gaia EDR3 4302054339959905920',
 'Gaia EDR3 1222646935698492160',
 'Gaia EDR3 5111187420714898304',
 'Gaia EDR3 1563590579347125632',
 'Gaia EDR3 4049506483413484672',
 'Gaia EDR3 6227443304915069056',
 'Gaia EDR3 804753180515722624',
 'Gaia EDR3 4429785739602747392',
 'Gaia EDR3 2106630885454013184',
 'Gaia EDR3 856096765753549056',
 'Gaia EDR3 3704342295607157120',
 'Gaia EDR3 4473334474604992384',
 'Gaia EDR3 3501215734352781440',
 'Gaia EDR3 2202630001603369856',
 'Gaia EDR3 702343774145932544',
 'Gaia EDR3 1625209684868707328',
 'Gaia EDR3 4076915349846977664',
 'Gaia EDR3 1297565458994346240',
 'Gaia EDR3 1415230383034813824',
 'Gaia EDR3 2971266110612490624',
 'Gaia EDR3 4337352305320149760',
 'Gaia EDR3 5099455184968674560',
 'Gaia EDR3 3736865265441207424',
 'Gaia EDR3 2968097